# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

# Reading Data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meetnagadia/human-action-recognition-har-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/razan./.cache/kagglehub/datasets/meetnagadia/human-action-recognition-har-dataset/versions/1


In [7]:
import os

# Define the full path to the dataset folder
dataset_folder = os.path.join(path, "Human Action Recognition")

# List files inside this folder
dataset_files = os.listdir(dataset_folder)
print("Files inside 'Human Action Recognition':", dataset_files)

Files inside 'Human Action Recognition': ['test', 'Testing_set.csv', 'Training_set.csv', 'train']


In [8]:
import pandas as pd
import os

# Define file paths
train_csv_path = os.path.join(path, "Human Action Recognition", "Training_set.csv")
test_csv_path = os.path.join(path, "Human Action Recognition", "Testing_set.csv")

# Load CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Display first few rows
print("Training Data:")
print(train_df.head())

print("\nTesting Data:")
print(test_df.head())

Training Data:
      filename         label
0  Image_1.jpg       sitting
1  Image_2.jpg  using_laptop
2  Image_3.jpg       hugging
3  Image_4.jpg      sleeping
4  Image_5.jpg  using_laptop

Testing Data:
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg


In [9]:
train_folder = os.path.join(path, "Human Action Recognition", "train")
test_folder = os.path.join(path, "Human Action Recognition", "test")

# List files inside train and test folders
print("Train folder contents:", os.listdir(train_folder))
print("Test folder contents:", os.listdir(test_folder))

Train folder contents: ['Image_11832.jpg', 'Image_8240.jpg', 'Image_7173.jpg', 'Image_1502.jpg', 'Image_5764.jpg', 'Image_811.jpg', 'Image_3315.jpg', 'Image_3473.jpg', 'Image_5002.jpg', 'Image_12485.jpg', 'Image_1264.jpg', 'Image_9638.jpg', 'Image_8526.jpg', 'Image_7615.jpg', 'Image_10292.jpg', 'Image_11198.jpg', 'Image_1270.jpg', 'Image_8532.jpg', 'Image_7601.jpg', 'Image_10286.jpg', 'Image_3467.jpg', 'Image_4308.jpg', 'Image_5016.jpg', 'Image_2779.jpg', 'Image_12491.jpg', 'Image_5770.jpg', 'Image_805.jpg', 'Image_3301.jpg', 'Image_8254.jpg', 'Image_11826.jpg', 'Image_7167.jpg', 'Image_6279.jpg', 'Image_1516.jpg', 'Image_2037.jpg', 'Image_5758.jpg', 'Image_4446.jpg', 'Image_3329.jpg', 'Image_5980.jpg', 'Image_6251.jpg', 'Image_9162.jpg', 'Image_1258.jpg', 'Image_6537.jpg', 'Image_9604.jpg', 'Image_7629.jpg', 'Image_2989.jpg', 'Image_4320.jpg', 'Image_193.jpg', 'Image_2751.jpg', 'Image_4334.jpg', 'Image_187.jpg', 'Image_2745.jpg', 'Image_6523.jpg', 'Image_9610.jpg', 'Image_8268.jpg', '

In [36]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


# Load unique labels and create a label mapping dictionary
def get_label_mapping(csv_file):
    df = pd.read_csv(csv_file)
    unique_labels = df["label"].unique()
    return {label: idx for idx, label in enumerate(unique_labels)}

# Define dataset class
class HumanActionDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, train=True):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.train = train  # Differentiates train and test datasets

        # Create label mapping only for training data
        if self.train:
            self.label_mapping = get_label_mapping(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get image filename
        img_name = self.data.iloc[idx, 0]  # First column is filename
        img_path = os.path.join(self.img_dir, img_name)

        # Load image, handle missing files
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: Image {img_name} not found. Using a blank image.")
            image = Image.new("RGB", (224, 224), (0, 0, 0))  # Blank black image

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Get label (for training data)
        if self.train:
            label = self.data.iloc[idx, 1]  # Second column is label
            label = self.label_mapping[label]  # Convert to integer index
            return image, torch.tensor(label, dtype=torch.long)

        return image

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert to tensor
])

# Define paths
train_csv = os.path.join(path, "Human Action Recognition", "Training_set.csv")
test_csv = os.path.join(path, "Human Action Recognition", "Testing_set.csv")
train_img_dir = os.path.join(path, "Human Action Recognition", "train")
test_img_dir = os.path.join(path, "Human Action Recognition", "test")

# Create dataset instances
train_dataset = HumanActionDataset(csv_file=train_csv, img_dir=train_img_dir, transform=transform, train=True)
test_dataset = HumanActionDataset(csv_file=test_csv, img_dir=test_img_dir, transform=transform, train=False)

# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)


# Check dataset info
print(f"Training Samples: {len(train_dataset)}")
print(f"Testing Samples: {len(test_dataset)}")

# Check if labels are correctly encoded
for images, labels in train_loader:
    print("Sample Labels:", labels[:5])
    break


Training Samples: 12600
Testing Samples: 5400
Sample Labels: tensor([ 8,  5, 10,  9,  6])


In [37]:
print(f"Train Loader Size: {len(train_loader)}")
print(f"Test Loader Size: {len(test_loader)}")

Train Loader Size: 1575
Test Loader Size: 675


# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [38]:
# Get unique labels
unique_labels = train_df["label"].unique()
num_classes = len(unique_labels)

print("Unique Labels:", unique_labels)
print("Number of Unique Labels (Classes):", num_classes)

Unique Labels: ['sitting' 'using_laptop' 'hugging' 'sleeping' 'drinking' 'clapping'
 'dancing' 'cycling' 'calling' 'laughing' 'eating' 'fighting'
 'listening_to_music' 'running' 'texting']
Number of Unique Labels (Classes): 15


In [39]:
# Get the shape of the training and testing datasets
train_shape = train_df.shape
test_shape = test_df.shape

print("Training Data Shape:", train_shape)  # (samples, features)
print("Testing Data Shape:", test_shape)    # (samples, features)

Training Data Shape: (12600, 2)
Testing Data Shape: (5400, 1)


In [40]:
# Initialize variables to track min and max pixel values
max_pixel_value = float('-inf')
min_pixel_value = float('inf')

# Loop through a batch of images
for images, _ in train_loader:
    max_pixel_value = max(max_pixel_value, images.max().item())  # Max pixel value
    min_pixel_value = min(min_pixel_value, images.min().item())  # Min pixel value
    break  # Only checking one batch for speed

print(f"Maximum Pixel Value: {max_pixel_value}")
print(f"Minimum Pixel Value: {min_pixel_value}")

Maximum Pixel Value: 1.0
Minimum Pixel Value: 0.0


In [41]:
for img, lbl in train_loader:
    print("Image shape:", img.shape)
    print("Label:", lbl)
    break

Image shape: torch.Size([8, 3, 224, 224])
Label: tensor([ 7,  2, 13, 11,  0, 13,  7, 11])


# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense, Conv2D) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [42]:
import torch.nn as nn
import torch.optim as optim

# Define a 5-layer CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        # 1st Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces spatial size

        # 2nd Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()

        # Flatten Layer
        self.flatten = nn.Flatten()

        # Fully Connected Layer
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Adjust based on input size
        self.relu3 = nn.ReLU()
        
        # Output Layer
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))
        x = self.pool(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)  # No activation (handled by loss function)
        return x

# Get number of unique classes
num_classes = len(train_df["label"].unique())

# Instantiate the model
model = SimpleCNN(num_classes=num_classes)

print(model)


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=15, bias=True)
)


In [43]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=15, bias=True)
)

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5  # Reduce for faster testing
for epoch in range(num_epochs):
    print(f"\n🚀 Starting Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Synchronize for MPS (Macs)
        if device.type == "mps":
            torch.mps.synchronize()

        # Update loss & accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Print progress every 10 batches
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx+1}/{len(train_loader)} | Loss: {loss.item():.4f}")

    train_accuracy = 100 * correct / total if total > 0 else 0
    avg_loss = running_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} Finished - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

print("🎉 Training Complete!")


🚀 Starting Epoch 1/5
Batch 1/1575 | Loss: 2.6940
Batch 11/1575 | Loss: 2.7162
Batch 21/1575 | Loss: 2.6952
Batch 31/1575 | Loss: 2.7096
Batch 41/1575 | Loss: 2.7825
Batch 51/1575 | Loss: 2.6970
Batch 61/1575 | Loss: 2.7043
Batch 71/1575 | Loss: 2.6897
Batch 81/1575 | Loss: 2.8135
Batch 91/1575 | Loss: 2.7355
Batch 101/1575 | Loss: 2.7203
Batch 111/1575 | Loss: 2.9547
Batch 121/1575 | Loss: 2.6413
Batch 131/1575 | Loss: 2.7253
Batch 141/1575 | Loss: 2.7153
Batch 151/1575 | Loss: 2.7172
Batch 161/1575 | Loss: 2.7271
Batch 171/1575 | Loss: 2.6881
Batch 181/1575 | Loss: 2.7167
Batch 191/1575 | Loss: 2.7184
Batch 201/1575 | Loss: 2.6938
Batch 211/1575 | Loss: 2.7200
Batch 221/1575 | Loss: 2.6969
Batch 231/1575 | Loss: 2.7184
Batch 241/1575 | Loss: 2.6741
Batch 251/1575 | Loss: 2.7203
Batch 261/1575 | Loss: 2.6739
Batch 271/1575 | Loss: 2.7039
Batch 281/1575 | Loss: 2.6234
Batch 291/1575 | Loss: 2.5278
Batch 301/1575 | Loss: 2.3371
Batch 311/1575 | Loss: 2.6731
Batch 321/1575 | Loss: 2.5892

Batch 1111/1575 | Loss: 2.1107
Batch 1121/1575 | Loss: 2.4598
Batch 1131/1575 | Loss: 1.5977
Batch 1141/1575 | Loss: 2.6939
Batch 1151/1575 | Loss: 2.1013
Batch 1161/1575 | Loss: 1.9204
Batch 1171/1575 | Loss: 1.8070
Batch 1181/1575 | Loss: 2.3793
Batch 1191/1575 | Loss: 1.5598
Batch 1201/1575 | Loss: 2.3087
Batch 1211/1575 | Loss: 2.5452
Batch 1221/1575 | Loss: 1.9542
Batch 1231/1575 | Loss: 2.2949
Batch 1241/1575 | Loss: 1.6442
Batch 1251/1575 | Loss: 2.1929
Batch 1261/1575 | Loss: 1.5315
Batch 1271/1575 | Loss: 1.9015
Batch 1281/1575 | Loss: 1.7265
Batch 1291/1575 | Loss: 2.0096
Batch 1301/1575 | Loss: 1.5492
Batch 1311/1575 | Loss: 2.4712
Batch 1321/1575 | Loss: 2.3428
Batch 1331/1575 | Loss: 2.1727
Batch 1341/1575 | Loss: 2.3639
Batch 1351/1575 | Loss: 2.4366
Batch 1361/1575 | Loss: 2.6253
Batch 1371/1575 | Loss: 2.1958
Batch 1381/1575 | Loss: 1.9633
Batch 1391/1575 | Loss: 1.4622
Batch 1401/1575 | Loss: 1.8925
Batch 1411/1575 | Loss: 2.1989
Batch 1421/1575 | Loss: 2.6234
Batch 14

Batch 611/1575 | Loss: 0.7522
Batch 621/1575 | Loss: 0.9383
Batch 631/1575 | Loss: 0.3204
Batch 641/1575 | Loss: 0.7367
Batch 651/1575 | Loss: 1.0180
Batch 661/1575 | Loss: 0.3420
Batch 671/1575 | Loss: 0.3156
Batch 681/1575 | Loss: 0.0962
Batch 691/1575 | Loss: 0.1583
Batch 701/1575 | Loss: 0.3349
Batch 711/1575 | Loss: 0.2709
Batch 721/1575 | Loss: 0.0476
Batch 731/1575 | Loss: 0.6471
Batch 741/1575 | Loss: 0.0956
Batch 751/1575 | Loss: 0.2301
Batch 761/1575 | Loss: 0.6560
Batch 771/1575 | Loss: 0.2627
Batch 781/1575 | Loss: 0.1056
Batch 791/1575 | Loss: 0.3194
Batch 801/1575 | Loss: 0.9512
Batch 811/1575 | Loss: 0.4715
Batch 821/1575 | Loss: 1.0256
Batch 831/1575 | Loss: 0.4363
Batch 841/1575 | Loss: 0.3891
Batch 851/1575 | Loss: 0.6102
Batch 861/1575 | Loss: 0.1205
Batch 871/1575 | Loss: 0.4122
Batch 881/1575 | Loss: 1.2127
Batch 891/1575 | Loss: 1.0452
Batch 901/1575 | Loss: 0.3237
Batch 911/1575 | Loss: 0.4867
Batch 921/1575 | Loss: 1.3585
Batch 931/1575 | Loss: 0.5848
Batch 941/

In [48]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:  # Use a single variable instead of (images, labels)
        images = batch.to(device)  # Since test_loader only returns images
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # If you have true labels for testing, you should modify the dataset class to return labels
        # Otherwise, you can't compute accuracy

print("✅ Evaluation Complete!")

✅ Evaluation Complete!
